# About

_TODO_

_TODO_ Don't forget to add Graphulo jars to your server-side Accumulo files! More info here: _TODO_ 

# Accumulo

In [2]:
using Revise
using D4M

┌ Info: Precompiling D4M [ca196bdc-a701-11e8-3d50-3b5cc8577617]
└ @ Base loading.jl:1192


Database capabilities loaded!


In [3]:
testD4Mver()

v test 6


In [4]:
dbinit()

In [6]:
DB = dbsetup("uno", "AccumuloConfig.jl")

26 Jun 2019 09:29:56,648 WARN - ClientConfiguration.loadFromSearchPath(227) -  Found no client.conf in default paths. Using default client configuration values.


D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x00000000051639e8))

In [157]:
A = DB["mytable_A"]
AT = DB["mytable_AT"]
B = DB["mytable_B"]


ls(DB)

Creating mytable_A in uno
Creating mytable_AT in uno
Creating mytable_B in uno


8-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "idk"                 
 "mytable_A"           
 "mytable_AT"          
 "mytable_B"           
 "trace"               

In [284]:
#row = ["r1", "r1", "r2", "r2"]
row = ["1", "1", "2", "2"]
column = ["1","2","1","2"]
values = [1,2,3,4]

delete(A)
AssocA = Assoc(row,column,values)
put(A, AssocA)
printFull(A[:,:])
delete(Pfilter2)
delete(AT)
AssocAT = transpose(AssocA)
put(AT, AssocAT)
printFull(AT[:,:])

row = ["1", "1", "2", "2"]
column = ["1","2","1","2"]
values = [5,6,7,8]

delete(B)
AssocB = Assoc(row,column,values)
put(B, AssocB)
printFull(B[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "2"
 "2"  "3"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "3"
 "2"  "2"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "5"  "6"
 "2"  "7"  "8"

(for some reason, printFull prints the entries as strings... however, in Accumulo, they're still numbers and can be operated on, as we'll soon see)

# Table Multiplication

_TODO expand on this_
To multiply A and B, we pass in <b><u>A'</b></u> and B

In [286]:
delete(P)
P = tablemult(AT, B, "mytable_P")
printFull(P[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "19"  "22"
 "2"  "43"  "50"

27 Jun 2019 12:23:42,642 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 8 entries processed


Let's check this...

In [287]:
printFull(AssocA * AssocB)

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"  19     22   
 "2"  43     50   

We can add filters too. They are added as extra arguments to `tablemult`, and are formatted like D4M query strings (what you might put in a normal associative array filter). For example, here are some simple row filters:

In [289]:
delete(Pfilter1)
Pfilter1 = tablemult(AT, B, "mytable_Pfilter1", rowfilter="1,")
printFull(Pfilter1[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "5"   "6" 
 "2"  "15"  "18"

27 Jun 2019 12:24:26,500 DEBUG - Graphulo.TwoTable(770) -  1 :%00; [1] -> 4 entries processed


Table multiplication first applies row filters to AT, transposes it, and multiplies. We can check by using native D4M:

In [304]:
printFull(transpose(AssocAT["1,",:]) * AssocB["1,",:])

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"   5      6   
 "2"  15     18   

This is equivalent to running a column filter on the original matrix:

In [305]:
printFull(AssocA[:,1] * AssocB[1,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"   5      6   
 "2"  15     18   

As for column filters in `tablemult`:

In [306]:
delete(Pfilter2)
Pfilter2 = tablemult(AT, B, "mytable_Pfilter2", colfilterAT="1,")
printFull(Pfilter2[:,:])
printFull(transpose(AssocAT[:,"1,"]) * AssocB[:,:]) # to check

2×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "19"  "22"

27 Jun 2019 14:18:02,144 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 4 entries processed


2×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"  19     22   

In [307]:
Pfilter3 = tablemult(AT, B, "mytable_Pfilter3", colfilterB="1,")
printFull(Pfilter3[:,:])
printFull(transpose(AssocAT[:,:]) * AssocB[:,"1,"]) # to check

3×2 Array{Union{AbstractString, Number},2}:
 ""   "1" 
 "1"  "19"
 "2"  "43"

3×2 Array{Union{AbstractString, Number},2}:
 ""     "1"
 "1"  19   
 "2"  43   

27 Jun 2019 14:19:10,868 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 4 entries processed


Now, let's try some more extensive filters, with a bigger example. First, we create an associative array, and square it:

In [396]:
toDBstring([1,3,4,"d"])


4-element Array{Any,1}:
 1   
 3   
 4   
  "d"

In [397]:
isa([1,3,4,"d"], UnionArray)

false

In [342]:
row = "a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,a,aa,aaa,b,bb,bbb,aaa,b,bb,bbb,"
column = "a,aa,aaa,b,bb,bbb,a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,aa,aa,aa,aa,"
values = "a-a,a-aa,a-aaa,a-b,a-bb,a-bbb,a-a,aa-a,aaa-a,b-a,bb-a,bbb-a,a-a,aa-aa,aaa-aaa,b-b,bb-bb,bbb-bbb,a,a,a,a,"
AssocD = logical(Assoc(row,column,values))
printFull(AssocD)
AssocDT = transpose(AssocD)

delete(D)
D = DB["mytable_D"]
put(D, AssocD)
printFull(D[:,:])

delete(DT)
DT = DB["mytable_DT"]
put(DT, AssocDT)
printFull(DT[:,:])

7×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"    1     1      1       1     1      1     
 "aa"   1     1      0       0     0      0     
 "aaa"  1     1      1       0     0      0     
 "b"    1     1      0       1     0      0     
 "bb"   1     1      0       0     1      0     
 "bbb"  1     1      0       0     0      1     

Creating mytable_D in uno


7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "1"  "1"   "1"    "1"  "1"   "1"  
 "aa"   "1"  "1"   ""     ""   ""    ""   
 "aaa"  "1"  "1"   "1"    ""   ""    ""   
 "b"    "1"  "1"   ""     "1"  ""    ""   
 "bb"   "1"  "1"   ""     ""   "1"   ""   
 "bbb"  "1"  "1"   ""     ""   ""    "1"  

Creating mytable_DT in uno


7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "1"  "1"   "1"    "1"  "1"   "1"  
 "aa"   "1"  "1"   "1"    "1"  "1"   "1"  
 "aaa"  "1"  ""    "1"    ""   ""    ""   
 "b"    "1"  ""    ""     "1"  ""    ""   
 "bb"   "1"  ""    ""     ""   "1"   ""   
 "bbb"  "1"  ""    ""     ""   ""    "1"  

In [345]:
delete(D2)
D2 = tablemult(DT, D, "mytable_D2")
printFull(D2[:,:])
printFull(AssocD * AssocD)

7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "6"  "6"   "2"    "2"  "2"   "2"  
 "aa"   "2"  "2"   "1"    "1"  "1"   "1"  
 "aaa"  "3"  "3"   "2"    "1"  "1"   "1"  
 "b"    "3"  "3"   "1"    "2"  "1"   "1"  
 "bb"   "3"  "3"   "1"    "1"  "2"   "1"  
 "bbb"  "3"  "3"   "1"    "1"  "1"   "2"  

7×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"    6     6      2       2     2      2     
 "aa"   2     2      1       1     1      1     
 "aaa"  3     3      2       1     1      1     
 "b"    3     3      1       2     1      1     
 "bb"   3     3      1       1     2      1     
 "bbb"  3     3      1       1     1      2     

27 Jun 2019 20:17:07,422 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 72 entries processed


Let's try some filters:

Multiplying only a few rows and columns:

In [351]:
delete(D2f1)
D2f1 = tablemult(DT, D, "mytable_D2f1", rowfilter = "a,b,", colfilterAT = "a,bb,")
printFull(D2f1[:,:])
printFull(transpose(AssocDT["a,b,","a,bb,"]) * AssocD["a,b,",:])

3×7 Array{Union{AbstractString, Number},2}:
 ""    "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"   "2"  "2"   "1"    "2"  "1"   "1"  
 "bb"  "1"  "1"   "1"    "1"  "1"   "1"  

3×7 Array{Union{AbstractString, Number},2}:
 ""     "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"   2     2      1       2     1      1     
 "bb"  1     1      1       1     1      1     

27 Jun 2019 20:33:08,233 DEBUG - Graphulo.TwoTable(770) -  b :%00; [2] -> 15 entries processed


Multiplying ranges of rows and columns:

In [367]:
delete(D2f2)
D2f2 = tablemult(DT, D, "mytable_D2f2", rowfilter = "a,:,b,", colfilterB = "a,:,aaa,")
printFull(D2f2[:,:])
printFull(transpose(AssocDT["a,:,b,",:]) * AssocD["a,:,b,","a,:,aaa,"])

7×4 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"
 "a"    "4"  "4"   "2"  
 "aa"   "2"  "2"   "1"  
 "aaa"  "3"  "3"   "2"  
 "b"    "3"  "3"   "1"  
 "bb"   "2"  "2"   "1"  
 "bbb"  "2"  "2"   "1"  

7×4 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"
 "a"    4     4      2     
 "aa"   2     2      1     
 "aaa"  3     3      2     
 "b"    3     3      1     
 "bb"   2     2      1     
 "bbb"  2     2      1     

27 Jun 2019 20:39:09,358 DEBUG - Graphulo.TwoTable(770) -  a :%00; [1] -> 40 entries processed


Multiplying rows and columns that start with certain strings:

In [381]:
delete(D2f3)
D2f3 = tablemult(DT, D, "mytable_D2f3", colfilterAT = StartsWith("aaa,bb,"))
printFull(D2f3[:,:])
printFull(transpose(AssocDT[:,StartsWith("aaa,bb,")]) * AssocD[:,:])

4×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "aaa"  "3"  "3"   "2"    "1"  "1"   "1"  
 "bb"   "3"  "3"   "1"    "1"  "2"   "1"  
 "bbb"  "3"  "3"   "1"    "1"  "1"   "2"  

4×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "aaa"  3     3      2       1     1      1     
 "bb"   3     3      1       1     2      1     
 "bbb"  3     3      1       1     1      2     

27 Jun 2019 21:21:03,693 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 33 entries processed


In [384]:
isa([1,2], NumericArray)

true

In [383]:
    StringArray = Array{T} where T <: AbstractString
    NumericArray = Array{T} where T <: Number
    UnionArray = Array{T} where T <: Union{AbstractString,Number}

Array{T,N} where N where T<:Union{AbstractString, Number}